# OTROS TIEMPOS

In [31]:
weeks = 2

#52 semanas 8 minutos 
# 5 semanas 50 segundos

In [32]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "search",
    [[["inicio", ">=", FECHA_INICIO], ["inicio", "<=", FECHA_FIN]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "read",
    [lost_sales_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df = pd.DataFrame(lost_sales_data)

df['tarea'] = df['tarea_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

#Tarda  menos de 1 minuto

📅 Descargando registros desde 2025-10-17 hasta 2025-10-31


gaierror: [Errno -3] Temporary failure in name resolution

In [ ]:
df.head(2)

,create_uid,__last_update,wk_line_id,create_date,name,tipo,tarea_id,delay_pausa,is_pause,finalizo,delay_parcial,delay_total,production_id,inicio,write_date,workcenter_id,costo_total,write_uid,id,delay_hours,display_name,tarea
0,"[345, Planta1]",2025-09-26 11:37:11,False,2025-09-26 11:37:11,"[292, ALVARO MELO PRADA]",produccion,"[9, Reproceso]",0.0,False,2025-09-26 06:37:11,0.0,0.0,False,2025-09-26 06:37:11,2025-09-26 11:37:11,False,0.00,"[345, Planta1]",31864,0.0,ALVARO MELO PRADA,Reproceso
1,"[369, BRAYAN NAIR MUÑOZ SALAS]",2025-09-26 20:11:27,False,2025-09-26 11:41:40,"[553, KATHERIN ANDREA GIL AGUILAR]",produccion,"[12, Auxiliar producción]",0.0,False,2025-09-26 15:11:29,0.0,0.0,False,2025-09-26 06:41:40,2025-09-26 20:11:27,False,134175.72,"[1, Administrador Administrador ]",31865,8.5,KATHERIN ANDREA GIL AGUILAR,Auxiliar producción


In [ ]:
df_ot = df[df["tipo"]!="sistema"] #Se quitan las de op ya que no caen bien

df_ot.columns = df_ot.columns.str.lower()
df_ot = df_ot.rename(columns={
    "display_name": "operario",
    "inicio": "fecha_inicio",
    "tipo": "actividad",
})

# Convertir fechas
for col in ["fecha_inicio", "create_date", "__last_update"]:
    df_ot[col] = pd.to_datetime(df_ot[col], errors="coerce")

# Agregar columnas que no existen en df_ot para igualar estructura
df_ot["duracion_horas"] = df_ot["delay_hours"]
df_ot["centro_produccion"] = "OTRAS TAREAS"

df_ot = df_ot[["operario","__last_update","actividad","finalizo","fecha_inicio","tarea","duracion_horas"]]

# Hora actual (en tu zona horaria si quieres)
ahora = pd.Timestamp.now()

df_ot["fecha_inicio"] = pd.to_datetime(df_ot["fecha_inicio"], errors="coerce")
df_ot["finalizo"] = pd.to_datetime(df_ot["finalizo"], errors="coerce")

# --- 1️⃣ Crear columna en_curso ---
# True si no hay fecha de finalización
df_ot["en_curso"] = df_ot["finalizo"].isna() | (df_ot["finalizo"] == False)

# --- 2️⃣ Recalcular duración ---
df_ot["duracion_horas"] = df_ot.apply(
    lambda row: (
        (ahora - row["fecha_inicio"]).total_seconds() / 3600
        if row["en_curso"] and pd.notna(row["fecha_inicio"])
        else (row["finalizo"] - row["fecha_inicio"]).total_seconds() / 3600
        if pd.notna(row["finalizo"]) and pd.notna(row["fecha_inicio"])
        else 0
    ),
    axis=1
).round(2)

In [ ]:
df_ot.tail(5)

,operario,__last_update,actividad,finalizo,fecha_inicio,tarea,duracion_horas,en_curso
3269,INGRID CAROLINA GOMEZ DELPRADO,2025-10-31 13:39:21,produccion,2025-10-31 08:39:21,2025-10-31 07:34:15,Reproceso,1.08,False
3292,FRANCY CAROLINA PINZON CHACON,2025-10-31 14:18:18,produccion,2025-10-31 09:18:18,2025-10-31 09:06:00,Reproceso,0.20,False
3296,EDINSON PUPO SIERRA,2025-10-31 16:00:43,produccion,2025-10-31 11:00:43,2025-10-31 09:29:26,Alistamiento MP,1.52,False
3302,LUIS ENRIQUE MORA PEINADO,2025-10-31 15:03:27,produccion,2025-10-31 10:03:27,2025-10-31 09:46:44,Reproceso,0.28,False
3333,BLANCA DORA CUBIDES SANCHEZ,2025-10-31 17:51:36,produccion,2025-10-31 12:51:36,2025-10-31 12:32:14,Orden y Aseo,0.32,False


# TIEMPOS POR OP

In [ ]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd
import re

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

ops_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "search",
    [[["create_date", ">=", FECHA_INICIO], ["create_date", "<=", FECHA_FIN]]]
)

t_op = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "read",
    [ops_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df_op = pd.DataFrame(t_op)

df_op['Display_name'] = df_op['operario'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['OP'] = df_op['production_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['centro_produccion'] = df_op['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))


df_op["nom_filtro"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\(([^)]+)\)", x).group(1) if re.search(r"\(([^)]+)\)", x) else None
)

df_op["product_ref"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\[([A-Z0-9]+)\]", x).group(1) if re.search(r"\[([A-Z0-9]+)\]", x) else None
)

df_op["maquina"] = df_op["fuerza_lab_id"].astype(str).apply(
    lambda x: re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE).group(1).strip()
    if re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE)
    else None
)



df_op = df_op.drop(columns=["operario", "production_id", "workcenter_id","write_uid","id","display_name","create_uid","wc_line_id","product_fabricado_id"])

#Tarda aprox 4 minutos


📅 Descargando registros desde 2025-09-26 hasta 2025-10-31


In [ ]:
# --- Limpieza df_op ---
df_op.columns = df_op.columns.str.lower()
df_op = df_op.rename(columns={"evento": "actividad", "display_name": "operario"})

# Asegurar tipos
for col in ["fecha_evento", "create_date", "write_date", "__last_update"]:
    df_op[col] = pd.to_datetime(df_op[col], errors="coerce")


In [ ]:
from datetime import timedelta

# Asegurar tipos de fecha correctos
df_op["fecha_evento"] = pd.to_datetime(df_op["fecha_evento"], errors="coerce")

# --- 1️⃣ Crear columna 'final' ---
df_op["final"] = df_op.apply(
    lambda row: row["fecha_evento"] + timedelta(seconds=row["segundos_evento"])
    if pd.notna(row["segundos_evento"]) else row["fecha_evento"],
    axis=1
)

# --- 2️⃣ Ordenar por operario y fecha_evento ---
df_op = df_op.sort_values(["operario", "fecha_evento"])

# --- 3️⃣ Marcar el último evento por operario ---
df_op["ultimo_evento"] = df_op.groupby("operario")["fecha_evento"].transform("max")

# --- 4️⃣ Crear columna en_curso ---
# En curso = el evento más reciente del operario y su actividad es "iniciar"
df_op["en_curso"] = (df_op["fecha_evento"] == df_op["ultimo_evento"]) & (df_op["actividad"] == "iniciar")

# --- 5️⃣ Filtrar: eliminar 'iniciar' antiguos (solo mantener el último 'iniciar' si está en curso) ---
df_op_filtrado = df_op[
    (df_op["actividad"] != "iniciar") |  # mantener todo lo que no sea iniciar
    (df_op["en_curso"])                  # o mantener el último iniciar de cada operario
].copy()

# --- 6️⃣ Limpieza final ---
df_op_filtrado = df_op_filtrado.drop(columns=["ultimo_evento"])


In [ ]:
#df_op_filtrado[df_op_filtrado["en_curso"]==True]
df_op_filtrado[(df_op_filtrado["operario"]=="JAMER PUPO SIERRA")& (df_op_filtrado["fecha_evento"]>"2025-10-21")].head(2)

,__last_update,horas_evento,create_date,fecha_evento,fuerza_lab_id,actividad,segundos_evento,write_date,operario,op,centro_produccion,nom_filtro,product_ref,maquina,final,en_curso
6272,2025-10-21 12:10:37,0.55,2025-10-21 12:10:37,2025-10-21 07:10:37,"[113786, ENTUBADORA 3 - DA8145 FILTRO AIRE EXCAVADORA RHINO REX210]",finalizar,1997.0,2025-10-21 12:10:37,JAMER PUPO SIERRA,MO13349,CORTE,025 DA8145,DAB08145025,ENTUBADORA 3,2025-10-21 07:43:54,False
6313,2025-10-21 12:43:35,0.52,2025-10-21 12:43:35,2025-10-21 07:43:35,"[113804, ENTUBADORA 3 - DA8169 FILTRO AIRE COSECHADORAS WORL]",finalizar,1864.0,2025-10-21 12:43:35,JAMER PUPO SIERRA,MO13352,CORTE,025 DA8169,DAB08169025,ENTUBADORA 3,2025-10-21 08:14:39,False


## UNIR DATAFRAMES

In [ ]:
#df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/control_operarios.xlsx")

In [ ]:
df_op_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5048 entries, 6 to 10051
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   __last_update      5048 non-null   datetime64[ns]
 1   horas_evento       5048 non-null   float64       
 2   create_date        5048 non-null   datetime64[ns]
 3   fecha_evento       5048 non-null   datetime64[ns]
 4   fuerza_lab_id      5048 non-null   object        
 5   actividad          5048 non-null   object        
 6   segundos_evento    5048 non-null   float64       
 7   write_date         5048 non-null   datetime64[ns]
 8   operario           5048 non-null   object        
 9   op                 5048 non-null   object        
 10  centro_produccion  5048 non-null   object        
 11  nom_filtro         5048 non-null   object        
 12  product_ref        5048 non-null   object        
 13  maquina            4197 non-null   object        
 14  final       

In [ ]:
df_ot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 517 entries, 0 to 3333
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   operario        517 non-null    object        
 1   __last_update   517 non-null    datetime64[ns]
 2   actividad       517 non-null    object        
 3   finalizo        516 non-null    datetime64[ns]
 4   fecha_inicio    517 non-null    datetime64[ns]
 5   tarea           517 non-null    object        
 6   duracion_horas  517 non-null    float64       
 7   en_curso        517 non-null    bool          
dtypes: bool(1), datetime64[ns](3), float64(1), object(3)
memory usage: 32.8+ KB


In [ ]:
# Copias de trabajo
df1 = df_op_filtrado.copy()
df2 = df_ot.copy()

# --- Homologar df_op_filtrado a estructura de df_ot ---
df1 = df1.rename(columns={
    'fecha_evento': 'fecha_inicio',
    'final': 'finalizo',
    'horas_evento': 'duracion_horas',
    'centro_produccion': 'tarea'
})

# --- Agregar columna de origen ---
df1['origen'] = 'produccion'
df2['origen'] = 'otras_tareas'

# --- Crear columna de segundos_evento ---
# En producción ya existe, pero renombramos para coherencia
if 'segundos_evento' not in df1.columns:
    df1['segundos_evento'] = df1['duracion_horas'] * 3600
# En tareas la calculamos desde duracion_horas
df2['segundos_evento'] = df2['duracion_horas'] * 3600

# --- Unir ambos sin perder datos ---
df_union = pd.concat([df1, df2], ignore_index=True, sort=False)

# --- Ordenar columnas ---
cols_base = [
    'operario', '__last_update', 'actividad', 'fecha_inicio', 'finalizo',
    'tarea', 'duracion_horas', 'segundos_evento', 'en_curso', 'origen'
]

cols_extra = [
    'op', 'fuerza_lab_id', 'maquina', 'nom_filtro', 'product_ref',
    'create_date', 'write_date'
]

cols_final = cols_base + [c for c in cols_extra if c in df_union.columns]
df_union = df_union[cols_final]


if 'tarea' in df_union.columns:
    df_union['tarea'] = df_union['tarea'].astype(str).str.upper()


# Tomar fecha y hora actual
ahora = pd.Timestamp.now()

# Filtrar filas en curso
mask_en_curso = df_union['en_curso'] == True

# Calcular diferencia de tiempo
df_union.loc[mask_en_curso, 'duracion_horas'] = (
    (ahora - df_union.loc[mask_en_curso, 'fecha_inicio']).dt.total_seconds() / 3600
)

df_union.loc[mask_en_curso, 'segundos_evento'] = (
    df_union.loc[mask_en_curso, 'duracion_horas'] * 3600
)

# Asegurar que la columna sea texto y eliminar espacios
df_union['op'] = df_union['op'].astype(str).str.strip()

df_union['operario'] = df_union['operario'].astype(str).str.strip()

df_union.loc[df_union['actividad'].str.lower().str.strip() == 'produccion', 'actividad'] = 'otras_tareas'

# Guardar a Excel
df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/operarios_tiempo_completo.xlsx", index=False)

#TARDA 1 MINUTO


In [ ]:
df_union[df_union["en_curso"]==True].head()

,operario,__last_update,actividad,fecha_inicio,finalizo,tarea,duracion_horas,segundos_evento,en_curso,origen,op,fuerza_lab_id,maquina,nom_filtro,product_ref,create_date,write_date
457,BLANCA CECILIA AVELLA ARIAS,2025-10-31 12:59:48,iniciar,2025-10-31 07:59:48,2025-10-31 07:59:48,INYECCION,5.315425,19135.528596,True,produccion,MO13393,"[114600, OMS - DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.]",OMS,025 DA2570,DAB02570025,2025-10-31 12:59:48,2025-10-31 12:59:48
637,BLANCA DORA CUBIDES SANCHEZ,2025-10-31 18:16:44,iniciar,2025-10-31 13:16:44,2025-10-31 13:16:44,EMBALAJE,0.033202,119.528596,True,produccion,MO13333,"[114828, MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA4653 FILTRO AIRE 2o. HITACHI]",None,025 DA4653,DAE04653025,2025-10-31 18:16:44,2025-10-31 18:16:44
1117,CLARA MARLEN OSPINA CASTRO,2025-10-31 16:18:19,iniciar,2025-10-31 11:18:19,2025-10-31 11:18:19,EMBALAJE,2.006813,7224.528596,True,produccion,MO13400,"[114805, EMBALAJE - DA2926 FILTRO AIRE DONSSON - INTERNATIONAL 4300 - 7400]",EMBALAJE,025 DA2926,DAB02926025,2025-10-31 16:18:19,2025-10-31 16:18:19
1195,DANILO PUPO SIERRA,2025-10-31 14:13:59,iniciar,2025-10-31 09:13:59,2025-10-31 09:13:59,PLISADO,4.079036,14684.528596,True,produccion,MO13417,"[114788, PLISADORA - DA2006 FILTRO AIRE CAMION FOTON AUMARK BJ1039]",PLISADORA,025 DA2006,DAR02006025,2025-10-31 14:13:59,2025-10-31 14:13:59
1328,DAVID ESTEBAN MALAGON CETINA,2025-10-31 17:16:37,iniciar,2025-10-31 12:16:37,2025-10-31 12:16:37,EMBALAJE,1.035147,3726.528596,True,produccion,MO13400,"[114821, EMBALAJE - DA2926 FILTRO AIRE DONSSON - INTERNATIONAL 4300 - 7400]",EMBALAJE,025 DA2926,DAB02926025,2025-10-31 17:16:37,2025-10-31 17:16:37


In [ ]:
#df_union[(df_union["operario"]=="JAMER PUPO SIERRA")& (df_union["fecha_inicio"]>"2025-10-21")].head(7)